In [1]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [2]:
# setup the parameters
# number of different M's
gradients = 8
# number of input values
vals = 3
iterations = 5000
learning_rate = 0.5

# maximum answer to be learnt
max = gradients**2 + vals

# defining function to make training data
def training_data():
    
    n = 0
    rows = 1
    # array which we are using as our x values 
    # in equation of linear line, y = Mx + b
    # it includes 1 extra value as this will be used as our labal
    x = np.arange(vals+1).astype(np.int32)
    # empty array to write our training data to
    y = np.array([])
    
    # loop so it cycles through every gradient and adds on different b's
    for i in range(gradients):
        # we loop it so it also creates lines with b's
        # the maximum values will always be gradients^2
        for e in range(max-vals*i):
            
            # append to y, x*n + e, where n = M and e = b
            # reshape so that each time it appends it creates a new line
            y = np.append(y, x*n+e).reshape(rows,vals+1)
            
            # increase number of rows to reshape it
            rows += 1
        
        # increase gradient by 1
        n+=1
    
    y = y.astype(np.int32)
    # return the training data
    # and number of lines to learn
    return(y,np.size(y,0))

# print the training data
print(training_data())

(array([[ 0,  0,  0,  0],
       [ 1,  1,  1,  1],
       [ 2,  2,  2,  2],
       ..., 
       [43, 50, 57, 64],
       [44, 51, 58, 65],
       [45, 52, 59, 66]], dtype=int32), 452)


In [3]:
# data = training data
# training lines = number of different lines
data,training_lines = training_data()

# the length is for when we convert the numbers into a binary array
# the array will be all zeros except one, which will be 1
# this will be the particle in this point in time
# each one is like a frame in a video
# this value is the size of the maximum value + 1
length = max+1

# the full length is the length of all the input frames stacked into one, 1d array
full_length = length*vals

# this function turns the data into the arrays explained above
def set_data():
    
    # there are two arrays, one for training data nad one for labels
    in_data = np.zeros([training_lines,vals,length])
    # the labels will be one-hot arrays
    lab = np.zeros([training_lines,1,length])
    
    # this sets the values specified in the training data to one
    for i in range(training_lines):
        # we need to set each individual input value
        for a in range(vals):
            # set the value to a 1
            in_data[i][a][data[i][a]] = 1
            
        # set the label value to a 1
        lab[i][0][data[i][vals]] = 1
        
    # here, we reshape it tto the full length 1d array
    in_data = in_data.reshape(training_lines,1,full_length)
    
    # return the data and labels
    return(in_data,lab)    

# print converted data
#print(set_data())

In [4]:
# we define the weights, biases and inputs
# this will be input training data
x = tf.placeholder(tf.float32, [None, full_length])
# weights and biases
W = tf.Variable(tf.zeros([full_length, length]))
b = tf.Variable(tf.zeros([length]))
# function which gives the out put of training data
y = tf.matmul(x, W) + b

# we will feed the labels in here
y_ = tf.placeholder(tf.float32, [None, length])

# configure the loss function, using cross entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# define the optimizer
#train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)


In [5]:
# I am using a GPU
# this line limits memory usage of the GPU to 0.4 when session is created
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

# create interactive session using the GPU line for above
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

# initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# now we run the learning loop
for _ in range(iterations):

    # set training data and labels
    x_data, y_data = set_data()
    # only use one lines data
    # chooses line data by evenly spreading data over iterations
    x_data = x_data[np.round(_//(iterations/(training_lines-0.)), 0).astype(np.int32)]
    y_data = y_data[np.round(_//(iterations/(training_lines-0.)), 0).astype(np.int32)]
    
    # run the training optimizer
    sess.run(train_step, feed_dict={x: x_data, y_: y_data})

    # print steps and error 20 times in total
    if _ % (iterations/20) == 0:
        print 'step', _, 'out of', iterations
        print 'error =', sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data})

step 0 out of 5000
error = 2.3118
step 250 out of 5000
error = 0.213955
step 500 out of 5000
error = 1.41275
step 750 out of 5000
error = 0.149835
step 1000 out of 5000
error = 0.405349
step 1250 out of 5000
error = 3.09523
step 1500 out of 5000
error = 0.255521
step 1750 out of 5000
error = 1.15096
step 2000 out of 5000
error = 0.182108
step 2250 out of 5000
error = 0.538194
step 2500 out of 5000
error = 3.59583
step 2750 out of 5000
error = 0.277763
step 3000 out of 5000
error = 1.45283
step 3250 out of 5000
error = 0.210351
step 3500 out of 5000
error = 0.600125
step 3750 out of 5000
error = 3.63638
step 4000 out of 5000
error = 0.373996
step 4250 out of 5000
error = 1.97619
step 4500 out of 5000
error = 0.238482
step 4750 out of 5000
error = 0.846523


In [6]:
# initialize array to record test values
test_line = np.array([])

print 'input values, the max answer is', max-1, 'and max gradient is', gradients-1
# loop to ask user for input values
for i in range(vals):
    print '\nvalue',i+1,'?'
    val = int(input())
    test_line = np.append(test_line, val)

test_line = test_line.astype(np.int32)
    
# define function to convert test data into shape and form
# of training data
def test_model():
    
    # this is the array we are writing to
    input_array = np.array([])
    
    # for each input value, write to the input array
    # with a 1 in the position
    for a in range(vals):
        test_array = np.zeros([1,length])
        test_array[0][test_line[a]] = 1
        # we write to array and shape so new line for each test value
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*length])
        
    # return array
    return(input_array)   

# print array
#test_model()

input values, the max answer is 66 and max gradient is 7

value 1 ?
3

value 2 ?
6

value 3 ?
9


In [7]:
# run the function to get output
# using new weights nad biases
probs = sess.run(y, feed_dict={x:test_model()})

# "true" answer, should work with all linear lines
answer = (test_line[1]-test_line[0])+test_line[vals-1]

if answer == np.argmax(probs):
    print 'Correct'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', np.argmax(probs)
    print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]
else:
    print 'Wrong'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', np.argmax(probs)
    print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]
    
    # print true answer and its probabilities
    print '\nNumerical answer =', answer
    print 'Probability of', answer, '=', probs[0][answer]
    

# print the probabilities
print '\n\n', probs

Correct

Learnt answer = 12
Probaility of 12 = 1.47778


[[-4.55359745 -4.65057087 -4.54642487 -2.58185291 -4.704072   -5.12132359
  -1.16276526 -6.14595604 -3.29876256 -1.62739193 -1.44989026 -3.68344045
   1.47777569 -3.65069246 -2.65160656 -2.96174765 -2.60492706 -4.48045158
   0.35684872 -4.15486574 -3.94342661 -0.79719913 -3.77331209 -3.01514339
  -0.64579952 -2.62557983 -2.63197136 -2.62030005 -2.62631607 -2.62804127
  -2.62743044 -2.62993383 -2.6356988  -2.633461   -2.62523198 -2.64066601
  -2.64740777 -2.64661598 -2.65011978 -2.65307617 -2.65616083 -2.66002679
  -2.66444397 -2.6679709  -2.6717279  -2.67115545 -2.6677115  -2.6692977
  -2.67081642 -2.66534185 -2.65285063 -2.66510439 -2.65923214 -2.65180492
  -2.65461588 -2.58934689 -2.59411263 -2.58565331 -2.57467079 -2.56105876
  -2.55311203 -2.21598244 -2.20822001 -2.09812975 -1.94062185 -1.67798042
  -1.61730087 -5.77465725]]
